In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, LeakyReLU, Dense, Flatten, Softmax, Input, Reshape
from tensorflow import keras
import numpy as np
import tensorflow as tf

2023-08-19 16:41:46.086760: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-19 16:41:46.381721: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-19 16:41:46.383325: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 16:41:48.531523: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
BATCH_SIZE = 2

In [47]:
class SignRecognition(tf.keras.Model):
    def __init__(self, frames, kernel_size=2, stride=2):
        super(SignRecognition, self).__init__()
        
        self.reshape0 = Reshape(target_shape=(82, 2, 128))
        
        self.conv1 = Conv2D(frames // 2, kernel_size=kernel_size, strides=stride)
        self.bn1 = BatchNormalization()
        self.leaky_relu = LeakyReLU(0.1)

        self.reshape1 = Reshape(target_shape=(41, frames // 2, 1))
        
        self.conv2 = Conv2D(1, kernel_size=kernel_size, strides=stride)
        self.bn2 = BatchNormalization()
        
        self.reshape2 = Reshape(target_shape=(20, 32))

        self.lin1 = Dense(128, activation="tanh")

        self.reshape3 = Reshape(target_shape=(128, 20))
        
    def call(self, x):
        x = self.reshape0(x)
        x = self.conv1(x)  # B, 41, 1, 64
        x = self.leaky_relu(self.bn1(x))
        x = self.reshape1(x)
        
        x = self.conv2(x)  # B, 1, 32, 20
        x = self.leaky_relu(self.bn2(x))
        x = self.reshape2(x)
        x = self.lin1(x)
        x = self.reshape3(x)
        return x


In [48]:
frames = 128 # Specify the number of frames
model = SignRecognition(frames)

In [49]:
# Example data (replace with your own data)
x_test = np.random.rand(1, 128, 82, 2)

predictions = model.predict(x_test)
predictions.shape

1/1 [==============================] - 0s 283ms/step


(1, 128, 20)